In [ ]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain_qdrant import QdrantVectorStore

import pandas as pd

In [ ]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
def create_qdrant_collection():
    qdrant_url = os.getenv("QDRANT_URL")
    qdrant_api_key = os.getenv("QDRANT_API_KEY")
    csv_path = "/home/pod/GiftRecommender/database/BigBasketProducts.csv"
    collection_name = "product_catalog"

    # Load and prepare documents
    df = pd.read_csv(csv_path)
    df = df.dropna(subset=["description"])
    docs = [
        Document(page_content=row["description"], metadata=row.to_dict())
        for _, row in df.iterrows()
    ]

    # Initialize embedding model
    embedding_model = OpenAIEmbeddings()

    # Create or update Qdrant collection
    QdrantVectorStore.from_documents(
        documents=docs,
        embedding=embedding_model,
        url=qdrant_url,
        api_key=qdrant_api_key,
        collection_name=collection_name,
        prefer_grpc=False,
        batch_size=50,        # optional performance tweak
        timeout=3600,  # optional timeout setting
    )
    print(f"Collection '{collection_name}' has been (re)created and populated.")

In [ ]:
create_qdrant_collection()